# Front End Visualization

In [36]:
import gmaps
import gmaps.datasets
import pandas as pd
import geopandas as gpd
import json
import geocoder
import geocoder
import numpy as np
from shapely.geometry import Point
from ipywidgets.embed import embed_minimal_html
import timeit

# set constants
main_path = '/Users/rita/Dropbox/InsightDataScience/Project'
api_key = open(main_path + '/config.py', 'r')
api_key = api_key.read().replace('api_key = ', '').replace('“','').replace('”','')

gmaps.configure(api_key=api_key) # Your Google API key

# get information
df = gpd.read_file(main_path + '/Results/nyc_blk_map_with_vals_for_front_end.shp')
df_filt = gpd.read_file(main_path + '/Results/nyc_blk_map_with_vals_for_front_end_FILTERED.shp')
with open(main_path + '/Data/Locations/Boroughs/BoroughBoundaries.geojson') as f:
    geometry = json.load(f)


new_york_coordinates = (40.75, -73.9)
heatmap_gradient = [
    (250, 185, 123, 0),
   'yellow',
    (249, 127, 77, 1),
    'red',
]

# define layers
geojson_layer = gmaps.geojson_layer(geometry, fill_color=(0,0,0,0), fill_opacity=1)
heatmap_layer = gmaps.heatmap_layer(
    df_filt[['latitude', 'longitude']], weights=df_filt['magnitude'],
    max_intensity=10, point_radius=0.00075, opacity=1, gradient=heatmap_gradient, 
    dissipating=False)


# add address
address = '111 3rd St Brooklyn'
address += "; NY"
address_coords = geocoder.google(address,
                             key=api_key)

address_point_df = pd.DataFrame(data={'geometry': [Point(address_coords.latlng[1], address_coords.latlng[0])]})
address_point_map = gpd.GeoDataFrame(address_point_df, geometry='geometry', crs=df.crs)
combined = gpd.tools.sjoin(address_point_map, df, how='left') 
address_dict = [{'input_address': address, 'nbhd': combined.loc[0, 'Neighborho'],
                 'this_val' : combined.loc[0, 'Rating'],
                 'location': (combined.loc[0, 'latitude'], combined.loc[0, 'longitude'])}]
info_box_template = """
<dl>
<dt>Address</dt><dd>{input_address}</dd>
<dt>Neighborhood</dt><dd>{nbhd}</dd>
<dt>Active?</dt><dd>{this_val}</dd>
</dl>
"""
address_info = [info_box_template.format(**address) for address in address_dict]
address_location = [address['location'] for address in address_dict]
marker_layer = gmaps.marker_layer(address_location, info_box_content=address_info,
                                 hover_text='Click for Information')

# make figure and display
fig = gmaps.figure(center=new_york_coordinates, zoom_level=10.5)
fig.add_layer(geojson_layer)
fig.add_layer(heatmap_layer)
fig.add_layer(marker_layer)
embed_minimal_html('widget-test.html', views=fig)

# timer test
start_time = timeit.default_timer()
print(timeit.default_timer() - start_time)

4.288800118956715e-05


In [38]:
# add address
address = '111 3rd St'
address_coords = geocoder.google(address,
                             key=api_key)

address_point_df = pd.DataFrame(data={'geometry': [Point(address_coords.latlng[1], address_coords.latlng[0])]})
address_point_map = gpd.GeoDataFrame(address_point_df, geometry='geometry', crs=df.crs)
combined = gpd.tools.sjoin(address_point_map, df, how='left') 